# Análisis Detallado de Presupuesto

Este notebook proporciona un análisis profundo del presupuesto, proyecciones y recomendaciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Cargar y Preparar Datos

In [ ]:
df = pd.read_csv('../datos/presupuesto.csv')
PRESUPUESTO_TOTAL = 10_000_000

# Calcular métricas derivadas
df['porcentaje_abonado'] = (df['abonado'] / df['valor_total'] * 100).fillna(0)
df['fecha_limite'] = pd.to_datetime(df['fecha_limite'], errors='coerce')

print(f"Total de ítems en presupuesto: {len(df)}")
print(f"Categorías: {df['categoria'].unique()}")

## 2. Desglose IGLESIA vs SALÓN

In [ ]:
comparacion = df.groupby('categoria').agg({
    'valor_total': 'sum',
    'abonado': 'sum',
    'saldo': 'sum',
    'item': 'count'
}).rename(columns={'item': 'num_items'})

comparacion['%_comprometido'] = (comparacion['valor_total'] / PRESUPUESTO_TOTAL * 100).round(2)
comparacion['%_abonado_categoria'] = (comparacion['abonado'] / comparacion['valor_total'] * 100).fillna(0).round(2)

print("\n" + "="*70)
print("           COMPARACIÓN IGLESIA vs SALÓN")
print("="*70 + "\n")

for categoria in comparacion.index:
    row = comparacion.loc[categoria]
    print(f"📍 {categoria}:")
    print(f"   Número de ítems:        {row['num_items']:.0f}")
    print(f"   Valor total:            ${row['valor_total']:,.0f}")
    print(f"   Abonado:                ${row['abonado']:,.0f}")
    print(f"   Saldo pendiente:        ${row['saldo']:,.0f}")
    print(f"   % del presupuesto:      {row['%_comprometido']:.1f}%")
    print(f"   % abonado (categoría):  {row['%_abonado_categoria']:.1f}%")
    print()

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Comparación de valores
categorias = comparacion.index
x = range(len(categorias))
width = 0.35

axes[0].bar([i - width/2 for i in x], comparacion['valor_total'], width, label='Comprometido', color='#3498db')
axes[0].bar([i + width/2 for i in x], comparacion['abonado'], width, label='Abonado', color='#2ecc71')
axes[0].set_ylabel('Valor (COP)', fontsize=12)
axes[0].set_title('Comparación de Gastos por Categoría', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(categorias)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Porcentaje del presupuesto
axes[1].barh(categorias, comparacion['%_comprometido'], color=['#FF6B6B', '#4ECDC4'])
axes[1].set_xlabel('% del Presupuesto Total', fontsize=12)
axes[1].set_title('Porcentaje del Presupuesto por Categoría', fontsize=14, fontweight='bold')
for i, v in enumerate(comparacion['%_comprometido']):
    axes[1].text(v, i, f' {v:.1f}%', va='center', fontsize=11)
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Análisis de Ítems Sin Confirmar

In [ ]:
sin_confirmar = df[df['confirmado'] == False].copy()

if len(sin_confirmar) > 0:
    print("\n⚠️  ÍTEMS SIN CONFIRMAR\n")
    print("="*70)
    
    for _, row in sin_confirmar.iterrows():
        print(f"\n📌 {row['item']} ({row['categoria']})")
        if row['valor_total'] > 0:
            print(f"   Valor: ${row['valor_total']:,.0f}")
            print(f"   Abonado: ${row['abonado']:,.0f}")
            print(f"   Saldo: ${row['saldo']:,.0f}")
        else:
            print(f"   ⚠️  SIN COTIZACIÓN")
        if pd.notna(row['fecha_limite']):
            print(f"   Fecha límite: {row['fecha_limite'].strftime('%Y-%m-%d')}")
        if row['proveedor']:
            print(f"   Proveedor: {row['proveedor']}")
    
    total_sin_confirmar = sin_confirmar['valor_total'].sum()
    print(f"\n💰 Total comprometido sin confirmar: ${total_sin_confirmar:,.0f}")
else:
    print("\n✅ Todos los ítems están confirmados!")

## 4. Proyección de Gastos

In [ ]:
total_comprometido = df['valor_total'].sum()
total_abonado = df['abonado'].sum()
total_saldo = df['saldo'].sum()
disponible = PRESUPUESTO_TOTAL - total_comprometido

print("\n" + "="*70)
print("           PROYECCIÓN DE GASTOS")
print("="*70 + "\n")

print(f"Presupuesto total:          ${PRESUPUESTO_TOTAL:>15,}")
print(f"Gastos comprometidos:       ${total_comprometido:>15,.0f}")
print(f"Ya abonado:                 ${total_abonado:>15,.0f}")
print(f"Saldo por pagar:            ${total_saldo:>15,.0f}")
print(f"Disponible para nuevos:     ${disponible:>15,.0f}")
print(f"\n% del presupuesto usado:    {(total_comprometido/PRESUPUESTO_TOTAL*100):>15.2f}%")

# Advertencias
if disponible < 0:
    print(f"\n🔴 ALERTA: Has excedido el presupuesto por ${abs(disponible):,.0f}")
elif disponible < PRESUPUESTO_TOTAL * 0.1:
    print(f"\n🟡 ADVERTENCIA: Solo queda {(disponible/PRESUPUESTO_TOTAL*100):.1f}% del presupuesto")

# Visualización de proyección
fig, ax = plt.subplots(figsize=(10, 6))

categorias_proj = ['Abonado', 'Por Pagar', 'Disponible']
valores_proj = [total_abonado, total_saldo, max(0, disponible)]
colores_proj = ['#2ecc71', '#f39c12', '#3498db']

ax.barh(categorias_proj, valores_proj, color=colores_proj)
ax.axvline(x=PRESUPUESTO_TOTAL, color='red', linestyle='--', linewidth=2, label='Límite Presupuesto')
ax.set_xlabel('Pesos COP', fontsize=12)
ax.set_title('Proyección de Gastos - Estado Actual', fontsize=14, fontweight='bold')

for i, v in enumerate(valores_proj):
    ax.text(v, i, f' ${v:,.0f}', va='center', fontsize=10)

ax.legend()
plt.tight_layout()
plt.show()

## 5. Comparación: Presupuestado vs Real

In [ ]:
# Mostrar todos los ítems con valores
items_con_valor = df[df['valor_total'] > 0].copy()

if len(items_con_valor) > 0:
    items_con_valor = items_con_valor.sort_values('valor_total', ascending=False)
    
    print("\n📊 DETALLE DE TODOS LOS GASTOS\n")
    print("="*90)
    print(f"{'Item':<25} {'Categoría':<10} {'Total':>15} {'Abonado':>15} {'Saldo':>15}")
    print("="*90)
    
    for _, row in items_con_valor.iterrows():
        estado = '✓' if row['confirmado'] else '⚠'
        print(f"{estado} {row['item']:<23} {row['categoria']:<10} ${row['valor_total']:>13,.0f} "
              f"${row['abonado']:>13,.0f} ${row['saldo']:>13,.0f}")
    
    print("="*90)
    print(f"{'TOTALES':<25} {'':<10} ${items_con_valor['valor_total'].sum():>13,.0f} "
          f"${items_con_valor['abonado'].sum():>13,.0f} ${items_con_valor['saldo'].sum():>13,.0f}")
else:
    print("\n⚠️  Aún no hay gastos registrados. Actualiza el archivo presupuesto.csv")

## 6. Recomendaciones de Ahorro

In [ ]:
print("\n💡 RECOMENDACIONES\n")
print("="*70)

# Ítems sin cotizar
sin_cotizar = df[(df['valor_total'] == 0) & (df['confirmado'] == False)]
if len(sin_cotizar) > 0:
    print(f"\n1. PRIORIDAD ALTA: Cotizar {len(sin_cotizar)} ítems pendientes:")
    for _, row in sin_cotizar.iterrows():
        print(f"   - {row['item']} ({row['categoria']})")

# Ítems con saldo alto
saldos_altos = df[df['saldo'] > df['valor_total'] * 0.7]
if len(saldos_altos) > 0:
    print(f"\n2. Ítems con saldo alto por pagar (>70%):")
    for _, row in saldos_altos[saldos_altos['valor_total'] > 0].iterrows():
        print(f"   - {row['item']}: ${row['saldo']:,.0f} pendiente")

# Estado general
if disponible > 0:
    print(f"\n3. ✅ Tienes ${disponible:,.0f} disponibles para ajustes o imprevistos.")
elif disponible < 0:
    print(f"\n3. 🔴 URGENTE: Necesitas reducir gastos o aumentar presupuesto en ${abs(disponible):,.0f}")

print("\n4. Recuerda confirmar fechas límite con proveedores.")
print("5. Mantén un colchón del 10% para imprevistos (~$1,000,000).")

## 7. Exportar Reporte a Excel

In [ ]:
# Crear reporte completo
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'../datos/reporte_presupuesto_{timestamp}.xlsx'

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Hoja 1: Detalle completo
    df.to_excel(writer, sheet_name='Detalle_Completo', index=False)
    
    # Hoja 2: Resumen por categoría
    comparacion.to_excel(writer, sheet_name='Resumen_Categoria')
    
    # Hoja 3: Items sin confirmar
    if len(sin_confirmar) > 0:
        sin_confirmar.to_excel(writer, sheet_name='Sin_Confirmar', index=False)
    
    # Hoja 4: Resumen ejecutivo
    resumen = pd.DataFrame({
        'Concepto': ['Presupuesto Total', 'Comprometido', 'Abonado', 'Saldo por Pagar', 'Disponible', '% Usado'],
        'Valor': [PRESUPUESTO_TOTAL, total_comprometido, total_abonado, total_saldo, disponible, 
                  f"{(total_comprometido/PRESUPUESTO_TOTAL*100):.2f}%"]
    })
    resumen.to_excel(writer, sheet_name='Resumen_Ejecutivo', index=False)

print(f"\n✅ Reporte exportado exitosamente: {filename}")